## Actividad Face Recognition

In [1]:
import numpy as np
from scipy.io import loadmat
from pybalu.feature_selection import clean, sfs
from pybalu.feature_transformation import normalize, pca
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
data = loadmat('set05-face-detection.mat')

In [3]:
X = data['f']
y = data['d']

Utilizamos el comando _train_test_split_ para dividir el set en training y testing, con el cuidado de usar _stratify_ para evitar que quede con cantidades desequilibradas de labels 1 y 2 en cada set.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

Limpiamos las features con _clean_, y aplicamos los mismos índices en training y en testing

In [5]:
clean_indexes = clean(X_train)

In [6]:
X_train_cleaned = X_train[:, clean_indexes]
X_test_cleaned = X_test[:, clean_indexes]

Las normalizamos, también cuidadndo de aplicar la misma normalización a ambos sets

In [7]:
X_train_normalized, a, b = normalize(X_train_cleaned)
X_test_normalized = X_test_cleaned * a + b

Realizamos una selección de 40 características usando SFS

In [8]:
N_FEATURES = 40
selected_indexes = sfs(X_train_normalized, y_train, n_features=N_FEATURES, method="fisher", show=True)

Selecting Features: 100%|██████████| 40.0/40.0 [00:05<00:00, 6.72 features/s]


In [9]:
X_train_selected = X_train_normalized[:, selected_indexes]
X_test_selected = X_test_normalized[:, selected_indexes]

Realizamos PCA pero en dos variantes, en primer lugar con _sklearn_ y luego a modo de verificación con la implementación de _pybalu_.

In [10]:
pca_sk = PCA(n_components=10)
X_train_pcaed = pca_sk.fit_transform(X_train_selected)
X_test_pcaed = pca_sk.transform(X_test_selected)

Para cada una de las variantes realizamos un clasificador KNN y vemos su score.

In [11]:
knn_sk = KNeighborsClassifier(n_neighbors=1)
knn_sk.fit(X_train_pcaed, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [12]:
knn_sk.score(X_test_pcaed, y_test)

0.9056603773584906

Acá se repite el procedimiento pero con _pybalu_ como se explicó antes.

In [13]:
X_train_pcaed_pb, _, transform, _, _ = pca(X_train_selected, n_components=10)
X_test_pcaed_pb = X_test_selected @ transform

In [14]:
knn_pb = KNeighborsClassifier(n_neighbors=1)
knn_pb.fit(X_train_pcaed_pb, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [15]:
knn_pb.score(X_test_pcaed_pb, y_test)

0.9056603773584906

Podemos apreciar que da resultados bastante buenos y que la implementación de _sklearn_ con la de _pybalu_ es equivalente. En cualquier caso, al ser un set relativamente pequeño, la división y la aleatoriedad del stratify utilizado para dividir los sets, incide altamente en el resultado, y en nuestras pruebas el score osciló entre 84 y 98%.